In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbn

In [2]:
def event_data(year):
    return(pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/Events_{}.csv'.format(year)))
# def player_data(year):
#     return(pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/PlayByPlay_{}/Players_{}.csv'.format(year)))
year = 2011
p1 = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/PlayByPlay_{}/Players_{}.csv'.format(year,year)

SyntaxError: unexpected EOF while parsing (<ipython-input-2-6c698c627e0d>, line 6)

 __[For detailed explanations of the datasets: Link to Kaggle](https://www.kaggle.com/c/mens-machine-learning-competition-2018/data)__

In [ ]:
#This file identifies the different college teams present in the dataset.
teams = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/Teams.csv')

In [ ]:
#This file identifies the different seasons included in the historical data,
#along with certain season-level properties.
seasons = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/Seasons.csv')

In [ ]:
# This file identifies the seeds for all teams in each NCAA® tournament, for all seasons of historical data.
seeds = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/NCAATourneySeeds.csv')


In [ ]:
#This file identifies the game-by-game results for many seasons of historical data, starting with the 1985 season
reg_season_cpt = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/RegularSeasonCompactResults.csv')

In [ ]:
#This file identifies the game-by-game NCAA® tournament results for all seasons of historical data.
tour_results = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/NCAATourneyCompactResults.csv')


In [ ]:
#This file provides team-level box scores for many regular seasons of historical data, starting with the 2003 season. 
seas_detail = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/RegularSeasonDetailedResults.csv')


In [ ]:
#This file provides team-level box scores for many NCAA® tournaments, starting with the 2003 season.
tour_detail = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/NCAATourneyDetailedResults.csv')


In [ ]:
#This file indicates the Division I conferences that have existed over the years since 1985.
conferences = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/Conferences.csv')


In [ ]:
#This file indicates the conference affiliations for each team during each season. 
#Some conferences have changed their names from year to year, and/or changed which teams are part of the conference.
#This file tracks this information historically.

team_conferences = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/TeamConferences.csv')


In [ ]:
#This file indicates the head coach for each team in each season, 
#including a start/finish range of DayNums to indicate a mid-season coaching change. 
coaches = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/DataFiles/TeamCoaches.csv')

# Most Succesfull Teams? By Number of Champs

In [ ]:
champ_count = tour_results[tour_results['DayNum'] == 154].WTeamID.value_counts()

champ_teams = teams[teams.TeamID.isin(list(champ_count.index))][['TeamID','TeamName']]
champ_teams['no_champs'] = champ_teams.TeamID.map(champ_count)
Champ_Teams = champ_teams.sort_values(by = 'no_champs', ascending = False)


# Most Succesfull Team? By Average seed:

In [ ]:
def TeamID_to_Tname(series_TeamID):
    series_teamname = teams.set_index('TeamName').TeamID.map(series_TeamID).sort_values(ascending = False)
    return series_teamname
def PlayerID_to_TeamID(list_PlayerID, season):
    team_player = pd.read_csv('/Users/user/Desktop/proje/EGMStudyGroup/NCAA/data/PlayByPlay_{}/Players_{}.csv'.format(season,season))
    teamID = team_player[team_player.isin(list_PlayerID)]
    return teamID

In [ ]:
seeds['Seed'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
seeds['Seed_Numeric'] = pd.to_numeric(seeds.Seed)
Seed_Means = seeds.groupby('TeamID').Seed_Numeric.mean();

In [ ]:
Seeds_Teams = teams[teams.TeamID.isin(list(Seed_Means.index))][['TeamID','TeamName']]
# Seeds_Teams['avg_seeds'] = Seeds_Teams.TeamID.map(Seed_Means)
# Seed_Teams_15 = Seeds_Teams.sort_values(by = 'avg_seeds', ascending = True)[0:16]
# Seed_Teams_15[['TeamName', 'avg_seeds']].plot.bar(x = 'TeamName')



In [ ]:
Seed_Best = seeds[seeds['Seed_Numeric']==1].TeamID.value_counts()
Seeds_Best_Teams = teams[teams.TeamID.isin(list(Seed_Best.index))][['TeamID','TeamName']]
Seeds_Best_Teams['No_first_seed'] = Seeds_Best_Teams.TeamID.map(Seed_Best)
Seeds_Best_Teams = Seeds_Best_Teams.sort_values( by = 'No_first_seed', ascending = False)[0:14]



In [ ]:
def create_game_stats(events):
    # Day number, winning team id and losing team id uniquely determines the game id
    col = events['DayNum'].astype(str) + "_" + events['WTeamID'].astype(str) + "_" + events['LTeamID'].astype(str)
    events['GameID'] = col
    # new column indicating the winning team
    events.EventTeamID = (events.WTeamID == events.EventTeamID)
    events.EventTeamID = events.EventTeamID.astype(int)

    g = events.groupby(['GameID', 'EventTeamID', 'EventType']).agg({'EventType': 'count'})
    us = g.unstack(['EventTeamID', 'EventType'], fill_value=0)

    return us.sort_index(axis=1)

In [ ]:
season_wins = reg_season_cpt['WTeamID']
s1 = season_wins.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

f = fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (15,15))
f11 = TeamID_to_Tname(s1)[0:17].plot.barh(ax = ax1)
f11.set_title('Wins Between 1983 -2017')
f12 = Seeds_Best_Teams[['TeamName', 'No_first_seed']].plot.barh(ax = ax2, x = 'TeamName')
f12.set_title('Best Seeds 1983 - 2017')
f21 = TeamID_to_Tname(Seed_Means).sort_values(ascending = True)[0:16].plot.bar(ax = ax3, x = 'TeamName')
f21.set_title('Avg Seed')
f22 = Champ_Teams[['no_champs','TeamName']].plot.bar(ax = ax4, x = 'TeamName')
f22.set_title('Number of Champs 1983 - 2017')

# Here we want to predict game result